In [ ]:
import geopandas as gpd
from datetime import datetime
# 打开源文件
for name in range(2000,2021):
    fire_gdf = gpd.read_file('Pro_vic/'+ str(name)+'.shp')
    # 清洗confidence为0的虚假火灾点
    st = fire_gdf[fire_gdf['confidence'] !=0]
    # 保留发生月份在1，2，12, 11月的火灾点
    st['acq_date']=st['acq_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    stmonth = st['acq_date'].apply(lambda x: x.month)
    st2 = st[(stmonth == 11) | (stmonth == 12) | (stmonth == 1) | (stmonth == 2)]
    st2['acq_date']=st2['acq_date'].apply(lambda x:x.strftime('%Y-%m-%d'))
    st2.to_file('sort'+ str(name)+'.shp',
            driver='ESRI Shapefile',
            encoding='utf-8')

In [ ]:
# 火灾情况分析一下
# 打开源文件
import geopandas as gpd
dates = []
for name in range(2000,2021):
    st = gpd.read_file('Pro_vic/'+ str(name)+'.shp')
    dates=dates+list(st.acq_date)

from datetime import datetime
import pandas as pd
import numpy as np
dates = [datetime.strptime(x, "%Y-%m-%d") for x in dates]
dates = pd.Series(dates)
df = pd.DataFrame(dates,columns=['date'])
df["year"] = df["date"].apply(lambda x: x.year)
df["month"] = df["date"].apply(lambda x: x.month)
mc = df.groupby(["month"]).count()
yc = df.groupby(["year"]).count()

dates.to_csv('dates.csv')

In [ ]:
# 火灾情况简单可视化
# 以下部分切换出docker环境，因为环境里没有字体，而我不想配
import pandas as pd
from datetime import datetime
df = pd.read_csv('dates.csv',index_col=0)
df.columns=['date']
# dates = [datetime.strptime(x, "%Y-%m-%d") for x in dates]
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df["year"] = df["date"].apply(lambda x: x.year)
df["month"] = df["date"].apply(lambda x: x.month)
mc = df.groupby(["month"]).count()
yc = df.groupby(["year"]).count()

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
plt.rc('font',family='Times New Roman')
figprops = dict(figsize=(12, 3), dpi=300)

fig = plt.figure(**figprops)
ax1 = fig.add_subplot(121)
ax1.bar(range(2000,2000+len(yc['month'])),yc['month'])
ax1.set_title('Annual Statistics')
ax1.set_xticks(range(2000,2020,3))

ax1.xaxis.set_major_formatter(mtick.FormatStrFormatter('%d'))

ax2 = fig.add_subplot(122)
bara1 = ax2.bar(range(1,13),mc['year'],color=['#90EE90'])
bara2 = ax2.bar(range(1,13),mc['year'],color=['#5F9EA0'])
bara3 = ax2.bar(range(1,13),mc['year'],color=['#F4A460'])
bara4 = ax2.bar(range(1,13),mc['year'],color=['#CD5C5C','#CD5C5C','#F4A460','#F4A460','#F4A460','#5F9EA0','#5F9EA0','#5F9EA0','#90EE90','#90EE90','#90EE90','#CD5C5C'])
ax2.set_title('Monthly Statistics')
ax2.set_xticks(range(1,13))
ax2.set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Agu','Sept','Oct','Nov','Dec'])
ax2.legend(handles=[bara4,bara3,bara2,bara1], labels=['DJF', 'MAM','JJA','SON'],loc='best')
plt.show()
fig.savefig('statistics.pdf',format='pdf')

In [ ]:
# 合并到一个shp中，并且清洗不必要的变量
st2 = gpd.read_file('sort2000.shp')
for name in range(2001,2021):
    fire_gdf = gpd.read_file('sort'+ str(name)+'.shp')
    st2 = st2.append(fire_gdf)
st = st2
a = list(st2.columns)
st = st.drop(columns=a[16:])
# st = st.drop(columns=a[-38:])
# st = st.drop(columns=a[0:2])
# a = list(st.columns)
# st = st.drop(columns=a[20:120])
# a = list(st.columns)
# st = st.drop(columns=a[-5:])
st['geometry']=st2['geometry']
st.to_file('all.shp',
        driver='ESRI Shapefile',
        encoding='utf-8')

In [ ]:
import numpy as np
import netCDF4 as nc
from osgeo import gdal,osr,ogr
names = ['dmc','dc','ffmc','isi']
names = ['fwi','fdi','kbdi']
for name in names:
    datafile = "meteo_vic/"+name+".nc"
    nc_data_obj = nc.Dataset(datafile)
    Lon = nc_data_obj.variables['lon'][:]
    Lat = nc_data_obj.variables['lat'][:]
    data = np.array(nc_data_obj.variables[name][:])
    b = np.mean(data,axis=0)
    #影像的左上角和右下角坐标
    LonMin,LatMax,LonMax,LatMin = [Lon.min(),Lat.max(),Lon.max(),Lat.min()] 

    #分辨率计算
    N_Lat = len(Lat) 
    N_Lon = len(Lon)
    Lon_Res = (LonMax - LonMin) /(float(N_Lon)-1)
    Lat_Res = (LatMax - LatMin) / (float(N_Lat)-1)

    driver = gdal.GetDriverByName('GTiff')
    out_tif_name = 'meteo_vic/'+name+'.tif'
    out_tif = driver.Create(out_tif_name,N_Lon,N_Lat,1,gdal.GDT_Float32) 

    # 设置影像的显示范围
    #-Lat_Res一定要是-的
    geotransform = (LonMin,Lon_Res, 0, LatMax, 0, -Lat_Res)
    out_tif.SetGeoTransform(geotransform)

    #获取地理坐标系统信息，用于选取需要的地理坐标系统
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326) # 定义输出的坐标系为"WGS 84"，AUTHORITY["EPSG","4326"]
    out_tif.SetProjection(srs.ExportToWkt()) # 给新建图层赋予投影信息

    #数据写出
    out_tif.GetRasterBand(1).WriteArray(b) # 将数据写入内存，此时没有写入硬盘
    out_tif.FlushCache() # 将数据写入硬盘
    out_tif = None # 注意必须关闭tif文件

In [ ]:
# 火灾点shapefile转tiff
import matplotlib.pyplot as plt
import geopandas as gpd
st2 = gpd.read_file('all.shp')
plt.figure()
src = rio.open('victiff/f_water.tif')
profile = src.profile
bounds = src.bounds
shape = src.shape
data = src.read(1)
xedges = np.linspace(bounds[0], bounds[2], shape[1])
yedges = np.linspace(bounds[1], bounds[3], shape[0])
H, xedges, yedges = np.histogram2d(st2.geometry.x,st2.geometry.y, bins=(shape[1],shape[0]))
# print(H)
H = H.T
H = np.flipud(H)
H[np.isnan(data)] = np.nan
X, Y = np.meshgrid(xedges, yedges)
# plt.pcolormesh(X, Y, H)
plt.imshow(H)
plt.colorbar()

with rio.open('victiff/occur.tif', 'w', **profile) as dst:
     dst.write(H, 1)

In [ ]:
# river和water合并
import rasterio as rio
import numpy as np
src_river = rio.open('victiff/f_rive.tif')
src_water = rio.open('victiff/f_wate.tif')
river = src_river.read(1)
water = src_water.read(1)
river[river==src_river.nodata] = np.nan
water[water==src_water.nodata] = np.nan
profile = src_water.profile
w = np.array([river,water])
a=np.min(w,0)

with rio.open('victiff/example.tif', 'w', **profile) as dst:
     dst.write(a, 1)

In [ ]:
# 读victiff 所有栅格
import numpy as np
import rasterio as rio
# import geopandas as gpd
# names = ['est_occur','dem','disest_r_p','disest_r_s','disest_railway','disest_w','evi','landcover','ndvi','nw','slope']



def read_Data(names):
    bands = []
    datas = []
    src = rio.open('victiff/pick_mask1.tif')
    data = src.read(1)
    data = data.astype(np.double)
    data[data==src.nodata] = np.nan
    emptys = np.isnan(data)
    for name in names:
        src = rio.open('victiff/'+name+'.tif')
        data = src.read(1)
        data = data.astype(np.double)
        data[data==src.nodata] = np.nan
        emptys[np.isnan(data)] = True
        # break
        
        bands.append(data)
        # data = data[~np.isnan(data)]
        # print(data.shape)
    #     datas.append(data)
    bands = np.array(bands)
    # datas = np.array(datas)
    for i in range(bands.shape[0]):
        band = bands[i,:,:]
        band[emptys] = np.nan
        bands[i,:,:] = band
        data = band[~np.isnan(band)]
        datas.append(data)
    datas = np.array(datas)
    return bands,datas,emptys

def vif_corr(datas,names,dropname = None):
    import pandas as pd
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    import numpy as np
    from statsmodels.tools.tools import add_constant
    import matplotlib.pyplot as plt, seaborn
    # X = datas#[1:,:]
    X = pd.DataFrame(datas.T,columns = names)
    if len(dropname) != 0:
        for dropn in dropname:
            X = X.drop(axis=1, columns=dropn)
    X2 =add_constant(X)

    # 当VIF<10,说明不存在多重共线性；当10<=VIF<100,存在较强的多重共线性，当VIF>=100,存在严重多重共线性
    VIF_list = [variance_inflation_factor(X2,i) for i in range(X2.shape[1])]
    for VIF in VIF_list:
        print(VIF)
    df_corr = X.corr()
    fig = plt.figure(dpi=200)
    mask = np.zeros_like(df_corr)
    mask[np.triu_indices_from(mask)] = True
    seaborn.set(context='notebook', style='ticks', font_scale=0.5)
    with seaborn.axes_style("white"):
        ax = seaborn.heatmap(df_corr, center=0, annot=True, cmap="vlag",mask=mask, vmax=1,vmin=-1, square=True, xticklabels=X.columns, yticklabels=X.columns)
    plt.show()
    return VIF_list,df_corr,fig

def model_p_predict(datas,names,dropname = None):
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from statsmodels.formula.api import ols #加载ols模型
    from statsmodels.formula.api import poisson
    import matplotlib.pyplot as plt
    from statsmodels.tools.tools import add_constant
    from sklearn import metrics
    from sklearn.model_selection import train_test_split

    X3 = pd.DataFrame(datas.T,columns=names)
    if len(dropname) != 0:
        for dropn in dropname:
            X3 = X3.drop(axis=1, columns=dropn)
    X3['forest'] = X3['landcover2'].apply(lambda x: 1 if x==1 else 0)
    X3['shrublands'] = X3['landcover2'].apply(lambda x: 1 if x==2 else 0)
    X3['savannas'] = X3['landcover2'].apply(lambda x: 1 if x==3 else 0)
    X3['grasslands'] = X3['landcover2'].apply(lambda x: 1 if x==4 else 0)
    X3['wetlands'] = X3['landcover2'].apply(lambda x: 1 if x==5 else 0)
    # X3['croplands'] = X3['landcover'].apply(lambda x: 1 if x==6 else 0)
    X3 = X3.drop(axis=1, columns='landcover2')
    # total auc
    grouptable = X3

    ntable = (grouptable-grouptable.min())/(grouptable.max()-grouptable.min())
    y = grouptable['occur']
    x = ntable.drop(axis=1, columns='occur')
    x = add_constant(x)
    selected = list(ntable.columns)
    selected.remove('occur')
    selected = list(selected)

    formula = "occur ~ {}".format(
                ' + '.join(selected))
    auc_ts = []
    nowpars = []
    alls = []
    auc0 = 0
    for i in range(10):
        x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,train_size=0.8)
        # 多元线性回归
        # model = sm.GLM(y_train,x_train,family=sm.families.Poisson())
        # model = sm.GLM(y_train,x_train,family=sm.families.Poisson())
        model = ols(formula,data=x_train)
        results = model.fit()
        paramet = results.params
        nowpars.append(paramet)
        pred = model.predict(paramet, x_test)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, pred, pos_label=2)
        auc = metrics.auc(fpr, tpr)
        auc_ts.append(results.aic)
        # alls = pd.concat([y,y_test])
        # alls[y_test.index] = pred
        all = model.predict(paramet, x)
        alls.append(all)
        if auc > auc0:
            b_model = results
        # break
    return alls, auc_ts, nowpars,b_model
def model_m_predict(datas,names,dropname = None):
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from statsmodels.formula.api import ols #加载ols模型
    from statsmodels.formula.api import poisson
    import matplotlib.pyplot as plt
    from statsmodels.tools.tools import add_constant
    from sklearn import metrics
    from sklearn.model_selection import train_test_split

    X3 = pd.DataFrame(datas.T,columns=names)
    if len(dropname) != 0:
        for dropn in dropname:
            X3 = X3.drop(axis=1, columns=dropn)
    X3['forest'] = X3['landcover2'].apply(lambda x: 1 if x==1 else 0)
    X3['shrublands'] = X3['landcover2'].apply(lambda x: 1 if x==2 else 0)
    X3['savannas'] = X3['landcover2'].apply(lambda x: 1 if x==3 else 0)
    X3['grasslands'] = X3['landcover2'].apply(lambda x: 1 if x==4 else 0)
    X3['wetlands'] = X3['landcover2'].apply(lambda x: 1 if x==5 else 0)
    # X3['croplands'] = X3['landcover'].apply(lambda x: 1 if x==6 else 0)
    X3 = X3.drop(axis=1, columns='landcover2')
    # total auc
    grouptable = X3

    ntable = (grouptable-grouptable.min())/(grouptable.max()-grouptable.min())
    y = grouptable['occur']
    x = ntable.drop(axis=1, columns='occur')
    x = add_constant(x)
    auc_ts = []
    nowpars = []
    alls = []
    auc0 = 0
    for i in range(10):
        x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,train_size=0.8)
        # possion回归
        # model = sm.GLM(y_train,x_train,family=sm.families.Poisson())
        model = sm.GLM(y_train,x_train,family=sm.families.Poisson())
        results = model.fit()
        paramet = results.params
        nowpars.append(paramet)
        pred = model.predict(paramet, x_test, linear=False)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, pred, pos_label=2)
        auc = metrics.auc(fpr, tpr)
        auc_ts.append(results.aic)
        # alls = pd.concat([y,y_test])
        # alls[y_test.index] = pred
        all = model.predict(paramet, x, linear=False)
        alls.append(all)
        if auc > auc0:
            b_model = results
        # break
    return alls, auc_ts, nowpars,b_model
def save_figs(alls,emptys,name):
    import matplotlib.pyplot as plt
    import numpy as np
    import rasterio as rio
    alls = np.array(alls)
    src = rio.open('victiff/f_roads.tif')
    band = np.full(src.shape, np.nan)
    profile = src.profile
    a = np.mean(alls,axis=0)
    band[~emptys] = a
    plt.imshow(band)
    plt.colorbar()
    with rio.open('outputs/mean'+name+'.tif', 'w', **profile) as dst:
        dst.write(band, 1)
    b = np.std(alls,axis=0)
    band[~emptys] = b
    with rio.open('outputs/std'+name+'.tif', 'w', **profile) as dst:
        dst.write(band, 1)

In [ ]:
# 主程序
# ,'Pick_dmc','Pick_ffmc','Pick_isi'
# ,'Pick_dmc','Pick_ffmc','Pick_isi','Pick_fwi','Pick_fdi','Pick_kbdi'
names = ['occur','Pick_dem','f_roads','f_roadp','f_rail','f_water','f_evi','f_ndvi','Pick_nw','Pick_slope','landcover2','f_wui','Pick_dc']
# 读取数据
bands,datas,emptys = read_Data(names)
# 检查多元共线性
VIF_list,df_corr,fig_corr = vif_corr(datas, names, dropname =['landcover2'])
# 预测
alls, auc_ts, nowpars, b_model = model_p_predict(datas,names, dropname =['f_evi','Pick_slope'])
alls2, auc_ts2, nowpars2, b_model2 = model_m_predict(datas,names, dropname =['f_evi','Pick_slope'])
save_figs(alls2, emptys, '_ols')
save_figs(alls, emptys, '_pos')

In [ ]:
# 整理AIC summary
import pandas as pd
stats = [np.mean(auc_ts),np.min(auc_ts),np.max(auc_ts),np.max(auc_ts)-np.min(auc_ts),np.median(auc_ts),np.std(auc_ts),np.var(auc_ts)]
stats = pd.DataFrame(stats,index=['MEAN','MIN','MAX','RANGE','MEDIAN','STD','VAR'])
stats.to_csv('stats5.csv')

In [ ]:
# 折刀估计
def estimate_jack(datas,names):
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from statsmodels.formula.api import ols #加载ols模型
    from statsmodels.formula.api import poisson
    import matplotlib.pyplot as plt
    from statsmodels.tools.tools import add_constant
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    import statsmodels.formula.api as smf
    from statsmodels.formula.api import ols
    p_w = []
    p_o = []
    p_o2=[]
    p_w2=[]
    names3 = names[1:]
    names3.remove('f_evi')
    names3.remove('Pick_slope')
    names3.remove('landcover2')
    X3 = pd.DataFrame(datas.T,columns=names)
    X3 = X3.drop(axis=1, columns='Pick_slope')
    X3 = X3.drop(axis=1, columns='f_evi')
    X3 = X3.drop(axis=1, columns='landcover2')
    # total auc
    grouptable = X3
    y = grouptable['occur']
    x = grouptable.drop(axis=1, columns='occur')
    xp = add_constant(x)
    selected = list(grouptable.columns)
    selected.remove('occur')
    selected = list(selected)

    formula = "occur ~ {}".format(
            ' + '.join(selected))

    model = sm.GLM(y,xp,family=sm.families.Poisson())
    results = model.fit()
    p_t = results.aic

    model2 = ols(formula,data=grouptable)
    results2 = model2.fit()
    p_t2 = results2.aic
    for name in names3:
        grouptable = X3
        x2 = pd.DataFrame(grouptable[name])
        grouptable = grouptable.drop(axis=1, columns=name)
        y = grouptable['occur']
        x2['occur'] = y
        x = grouptable
        xp = add_constant(x)
        xp2 = add_constant(x2)
        
        # auc without 
        selected = list(grouptable.columns)
        selected.remove('occur')
        selected = list(selected)

        formula = "occur ~ {}".format(
                ' + '.join(selected))
        model = sm.GLM(y,xp,family=sm.families.Poisson())
        results = model.fit()
        p_w.append(results.aic)

        model2 = ols(formula,data=x)
        results2 = model.fit()
        p_w2.append(results.aic)

        # auc with only
        formula = "occur ~ {}".format(name)
        model = sm.GLM(y,xp2,family=sm.families.Poisson())
        results = model.fit()
        p_o.append(results.aic)

        model2 = ols(formula,data=x)
        results2 = model.fit()
        p_o2.append(results.aic)
    return p_t,p_t2,p_o,p_w,p_o2,p_w2
def draw_jack(p_t,p_t2,p_o,p_w,p_o2,p_w2,name=False):
    import numpy as np
    import matplotlib.pyplot as plt
    ap = [p_o,p_w,p_o2,p_w2]
    size = 20
    x = np.array(range(0,size,2))

    total_width, n = 2, 5
    width = total_width / n
    x = x - (total_width - width) / 2
    plt.Figure(dpi=300,figsize=(8,6))
    plt.barh(x[-1]  + 1.5 * width, p_t,  height=width,color='#DCDCDC' ,ls='-',ec='#000000',hatch='/')
    plt.barh(x[-1]  + 2.5 * width, p_t2,  height=width,color='#DCDCDC',ls='-',ec='#000000')
    plt.barh(x[:-1] + 0.5 * width, p_o, height=width, label='PR AIC with only',color='#FFFFFF',ls='-',ec='#000000',hatch='/')
    plt.barh(x[:-1] + 1.5 * width, p_w, height=width, label='PR AIC without',color='#A9A9A9',ls='-',ec='#000000',hatch='/')
    plt.barh(x[:-1] + 2.5 * width, p_o2, height=width, label='MLR AIC with only',color='#FFFFFF',ls='-',ec='#000000')
    plt.barh(x[:-1] + 3.5 * width, p_w2, height=width, label='MLR AIC without',color='#A9A9A9',ls='-',ec='#000000')
    plt.legend(bbox_to_anchor=(1.05,1.0),borderaxespad = 0.)
    plt.yticks(range(0,size,2),['Elevation','DRS','DRP','DRR','DW','NDVI','NW','DWUI','DC','Overall'])
    if name != False:
        plt.savefig(name+'.pdf',format='pdf')
    plt.show()

p_t,p_t2,p_o,p_w,p_o2,p_w2 = estimate_jack(datas,names)
draw_jack(p_t,p_t2,p_o,p_w,p_o2,p_w2,name='AIC')